Load Packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

Load Data

In [2]:
df = pd.read_excel('food_affordability.xls', sheet_name='Food_afford_cdp_co_region_ca')
df = df.drop(df.index[-1]) # remove last row because it is not part of the data

# Feature Engineering

In [3]:
# make factors 
df['region_code'] = df['region_code'].astype('category')
df['race_eth_code'] = df['race_eth_code'].astype('category')
df['geotypevalue'] = df['geotypevalue'].astype('category')
df['county_fips'] = df['county_fips'].astype('category')

Create features with zip code and lat/lon

In [4]:
all_zipcodes = pd.read_csv('zip_code_database.csv')

# names of all counties in the dataset
counties = ['Alameda', 'Alpine', 'Amador', 'Butte', 'Calaveras', 'Colusa',
       'Contra Costa', 'Del Norte', 'El Dorado', 'Fresno', 'Glenn',
       'Humboldt', 'Imperial', 'Inyo', 'Kern', 'Kings', 'Lake', 'Lassen',
       'Los Angeles', 'Madera', 'Marin', 'Mariposa', 'Mendocino',
       'Merced', 'Modoc', 'Mono', 'Monterey', 'Napa', 'Nevada', 'Orange',
       'Placer', 'Plumas', 'Riverside', 'Sacramento', 'San Benito',
       'San Bernardino', 'San Diego', 'San Francisco', 'San Joaquin',
       'San Luis Obispo', 'San Mateo', 'Santa Barbara', 'Santa Clara',
       'Santa Cruz', 'Shasta', 'Sierra', 'Siskiyou', 'Solano', 'Sonoma',
       'Stanislaus', 'Sutter', 'Tehama', 'Trinity', 'Tulare', 'Tuolumne',
       'Ventura', 'Yolo', 'Yuba']

# match my counties with those in the zipcode database
city_zipcodes = all_zipcodes[all_zipcodes['primary_city'].isin(counties)]
features_to_join = city_zipcodes[['latitude', 'longitude', 'primary_city']]
features_to_join['county_name'] = features_to_join['primary_city']
features_to_join = features_to_join.drop('primary_city', axis = 1)

# merge datasets
df = pd.merge(df, features_to_join, on = 'county_name', how = 'outer')

/var/folders/8f/l9_yqvn54_ldh6kclbdcgvfh0000gn/T/ipykernel_31276/73613974.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_to_join['county_name'] = features_to_join['primary_city']


In [5]:
# get all the numeric/categorical features together 
numeric_predictors = df.select_dtypes(include=['number']).drop('affordability_ratio', axis = 1) # df
numeric_predictors = numeric_predictors.drop(numeric_predictors.filter(regex='^is').columns, axis=1)
numeric_predictors = numeric_predictors.drop(columns = 'CA_RR_Affordability')
num_pred_names = numeric_predictors.columns

categorical_predictors = df.select_dtypes(include=['object', 'category']) # df
dummy_vars = df.filter(regex='^is')  # assuming dummy variables start with "is"
categorical_predictors = pd.concat([categorical_predictors, dummy_vars], axis=1)
cat_pred_names = categorical_predictors.columns

# consistent types for numeric columns
df[num_pred_names] = df[num_pred_names].apply(pd.to_numeric, errors='coerce')
# consistent types for categorical columns
df[cat_pred_names] = df[cat_pred_names].astype(str)

demographics features: Diversity Index

In [15]:
# using Simpson's Diversity Index
# measures the diversity of a dataset in the context of demographic studies
race_counts = df.groupby(['geoname', 'race_eth_name']).size().unstack(fill_value=0)
race_proportions = race_counts.div(race_counts.sum(axis=1), axis=0)

# Compute Simpson's Diversity Index
df['diversity_index'] = 1 - (race_proportions**2).sum(axis=1)

In [22]:
df['diversity_index'].notnull().sum()

0

household-level affordability

In [ ]:
df['affordability_per_person'] = df['affordability_ratio'] / df['ave_fam_size']

proportion of a specific racial group in each region

Final dataframe to csv

In [6]:
df.to_csv('final_data.csv', index = False)

# Supervised Machine Learning Models

In [7]:
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SelectPercentile, f_regression
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost.sklearn import XGBRegressor

### Data Preparation

In [8]:
df['median_income'] = df['median_income'].fillna(df['median_income'].mean()) # make sure response has no NAs

# get my X and Y matrices
X = df.drop(columns = ['median_income', 'version', 'ind_id', 'ind_definition', 'reportyear'])
y = df['median_income']

In [9]:
# extract only the column names for each type
numeric_features = X.select_dtypes(include = ['number']).columns
categorical_features = X.select_dtypes(include = ['object']).columns

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=307) # split the data

# make transformers for each data type
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  
    ('poly', PolynomialFeatures(include_bias=False)),
    ('scaler', StandardScaler())                  
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ('filtering', SelectPercentile(f_regression, percentile=50))     
])

# run both through the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),      
        ('cat', categorical_transformer, categorical_features)  
    ])

In [30]:
# may need to use samples for testing out some models
X_train_sample = X_train.sample(n=50000, random_state=42)
y_train_sample = y_train.loc[X_train_sample.index]

# scaling response - do this to full train/test when I am done with the samples
scaler = StandardScaler()
y_train_scaled = scaler.fit_transform(y_train_sample.values.reshape(-1, 1))
y_test_scaled = scaler.transform(y_test.values.reshape(-1, 1))

# taking the log of income may make most sense
y_train_log = np.log1p(y_train_sample)
y_test_log = np.log1p(y_test)


In [31]:
# looking at how my predictors are correlated with the response 
numeric_X_train = X_train_sample.select_dtypes(include=['number'])
corr = numeric_X_train.corrwith(y_train_log)
print(corr.sort_values(ascending=False))

food_afford_decile          0.881916
longitude                   0.003219
latitude                   -0.051667
cost_yr                    -0.053089
ave_fam_size               -0.083369
rse_food_afford            -0.241342
se_food_afford             -0.282512
LL95_affordability_ratio   -0.325106
UL95_affordability_ratio   -0.327396
affordability_ratio        -0.789804
CA_RR_Affordability        -0.789804
dtype: float64


### KNeighbors Regressor

In [14]:
# sample KNN Regressor
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', KNeighborsRegressor(n_neighbors=5, weights='distance'))  
])
pipe.fit(X_train_sample, y_train_log)
y_test_pred = pipe.predict(X_test)
test_mse = mean_squared_error(y_test_log, y_test_pred)
print(test_mse) #0.020348175708673553

0.0015055189271422165


### Linear Regression

In [72]:
# sample Linear Regression
lr_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())  
])
lr_pipe.fit(X_train_sample, y_train_sample)
y_test_pred = pipe.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
print(test_mse) # 0.16282604899371717 scaled


121894.90814926414


In [54]:
# evaluate model using cross-validation
scores = cross_val_score(lr_pipe, X_train_sample, y_train_scaled, scoring='neg_mean_squared_error', cv=5)
print(f"Average MSE: {-scores.mean()}")

Average MSE: 40475753.25730197


In [68]:
# now test the model using grid search with ridge

ridge_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', Ridge())
])
param_grid = {
    'model__alpha': [0.01, 0.1, 1, 10, 100]  # Regularization strength for Ridge
}

# GridSearchCV
grid_search = GridSearchCV(ridge_pipe, param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train_sample, y_train_sample)

# Results
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation MSE: {-grid_search.best_score_}")

#Best Parameters: {'model__alpha': 0.1}
#Best Cross-Validation MSE: 40669920.493706405

Best Parameters: {'model__alpha': 0.1}
Best Cross-Validation MSE: 40669920.493706405


In [69]:
# now test the model using grid search with lasso - takes a lot longer than Ridge

lasso_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', Lasso())
])
param_grid = {
    'model__alpha': [0.01, 0.1, 1, 10, 100]  # Regularization strength for Lasso
}

# GridSearchCV
grid_search = GridSearchCV(lasso_pipe, param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train_sample, y_train_sample)

# Results
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation MSE: {-grid_search.best_score_}")

/Users/madisonwozniak/miniconda3/envs/stat486/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1154199648359.4016, tolerance: 1004817349.5158395
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/madisonwozniak/miniconda3/envs/stat486/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1207070311069.476, tolerance: 1014963643.0435451
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/madisonwozniak/miniconda3/envs/stat486/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1184060532909.7727, tolerance: 988636664.5240126
  model = cd_fast.sparse_enet_coordina

KeyboardInterrupt: 

### Decision Trees/Random Forests

In [59]:
# make sure I don't have any nulls
X_train_sample.isnull().sum()
# since I do, I need to impute missing values

race_eth_code                   0
race_eth_name                   0
geotype                         0
geotypevalue                    0
geoname                         0
county_name                     0
county_fips                     0
region_name                     0
region_code                     0
cost_yr                      5114
affordability_ratio         32591
LL95_affordability_ratio    32997
UL95_affordability_ratio    32997
se_food_afford              32997
rse_food_afford             32997
food_afford_decile          45459
CA_RR_Affordability         32591
ave_fam_size                 2462
latitude                      616
longitude                     616
dtype: int64

In [63]:
# make new numeric transformer since I only want to impute now
dt_numeric_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy='mean'))
])

dt_preprocessor = ColumnTransformer(
    transformers=[
        ('num', dt_numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

pipe = Pipeline(steps=[
    ('preprocessor', dt_preprocessor),
    ('model', DecisionTreeRegressor(random_state = 42))  
])

pipe.fit(X_train_sample, y_train_sample) # do not use scaled versions because decision trees don't need that
y_test_pred = pipe.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
print(test_mse)

121894.90814926414


In [64]:
# tuning my decision tree

# using grid search
param_grid = {
    'model__max_depth': [3, 5, 10, None],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(pipe, param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train_sample, y_train_sample)
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best MSE: {-grid_search.best_score_}")

Best Parameters: {'model__max_depth': None, 'model__min_samples_leaf': 1, 'model__min_samples_split': 10}
Best MSE: 201151.66282008015


### Gradient Boosting (XGBoost, LightGBM)

In [71]:
# using XGBoost
xgb_model = XGBRegressor(n_estimators=100, 
                         learning_rate=0.1, 
                         max_depth=3, 
                         random_state=42,
                        )
xgb_model.fit(X_train_sample, y_train_sample)
# maybe add l1/l2 regularization 

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:race_eth_code: object, race_eth_name: object, geotype: object, geotypevalue: object, geoname: object, county_name: object, county_fips: object, region_name: object, region_code: object

In [ ]:
# evaluate model
y_test_pred = xgb_model.predict(X_test)
mse = mean_squared_error(y_test, y_test_pred)
print(f'Test MSE: {mse}')

In [ ]:
# graph feature importance
feature_importances = xgb_model.feature_importances_
plt.barh(range(len(feature_importances)), feature_importances)
plt.xlabel('Feature Importance')
plt.ylabel('Feature Index')
plt.show()

In [ ]:
# tune hyperparams with grid search
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0]
}

gb_grid_search = GridSearchCV(
    XGBRegressor(random_state=42),
    param_grid,
    scoring='neg_mean_squared_error',
    cv=5
)

gb_grid_search.fit(X_train_sample, y_train_sample)
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best MSE: {-grid_search.best_score_}")

In [ ]:
# graph feature importance
feature_importances = xgb_model.feature_importances_
plt.barh(range(len(feature_importances)), feature_importances)
plt.xlabel('Feature Importance')
plt.ylabel('Feature Index')
plt.show()

# Deep Learning Model with Hyperparam. Optimization

In [23]:
# load all DNN packages
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from keras.layers import LeakyReLU
import tensorflow as tf

In [32]:
X_train.shape

(221528, 20)

In [33]:
tf.random.set_seed(42)
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=[20]))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 20)                0         
                                                                 
 dense_3 (Dense)             (None, 300)               6300      
                                                                 
 dense_4 (Dense)             (None, 100)               30100     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 37,410
Trainable params: 37,410
Non-trainable params: 0
_________________________________________________________________


In [35]:
# compiling the model
model.compile(loss="sparse_categorical_crossentropy", #specify loss function - might need to change
              optimizer="adam",
              metrics=["accuracy"])           

In [39]:
# making train and validation subsets (221528 total)
X_trainANN, y_trainANN = X_train[:166146], y_train[:166146] 
X_validANN, y_validANN = X_train[-55382:], y_train[-55382:]

In [40]:
# training the model
n_epochs=10
history = model.fit(X_trainANN, y_trainANN, epochs=n_epochs,
                    validation_data=(X_validANN, y_validANN))

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

# Anomaly Detection or Cluster Analysis

# Dimension Reduction 

# SHAP for Feature Importance